## Human Pose Generation through Deformable GANs
- by Siarohin et al.

In [ ]:
import os
import torch
from torch.utils.data import DataLoader

# your modules
from dataset import MarketPoseDataset
from model.generator import DeformableGenerator
from model.discriminator import PatchDiscriminator
from train import train_model
from infer import infer_model
from evaluate import evaluate_model
from utils.visualization import tensor_to_image, save_comparison_grid
from utils.checkpoints import load_checkpoint, load_models_from_checkpoint

In [2]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
# ===== Cell 2: Configuration & Hyperparameters =====

# Data paths
images_dir = "data/1/Market-1501-v15.09.15/bounding_box_train"
pose_maps_dir = "data/processed/pose_maps"
train_pairs = "data/splits/train_pairs.csv"
val_pairs = "data/splits/val_pairs.csv"
test_pairs = val_pairs  # or a separate test set

# Training hyperparams
batch_size = 16
num_epochs = 20
lr = 2e-4
betas = (0.5, 0.999)
lambda_nn = 1.0
lambda_perceptual = 0.1
lambda_offset = 0.1
use_perceptual = True

# Output dirs
ckpt_dir = "outputs/ckpts"
sample_dir = "outputs/samples"
infer_out_dir = "outputs/infer"

os.makedirs(ckpt_dir, exist_ok=True)
os.makedirs(sample_dir, exist_ok=True)
os.makedirs(infer_out_dir, exist_ok=True)

In [ ]:
from torch.utils.data import Subset
import numpy as np

train_ds = MarketPoseDataset(images_dir, pose_maps_dir, train_pairs)
val_ds = MarketPoseDataset(images_dir, pose_maps_dir, val_pairs)
test_ds = MarketPoseDataset(images_dir, pose_maps_dir, test_pairs)

# Assuming train_dataset and val_dataset are already created
reduce_factor = 0.25  # use 25% of training data

num_train = len(train_ds)
num_val = len(val_ds)

subset_train_indices = np.random.choice(num_train, int(num_train * reduce_factor), replace=False)
subset_val_indices = np.random.choice(num_val, int(num_val * reduce_factor), replace=False)

train_dataset_small = Subset(train_ds, subset_train_indices)
val_dataset_small = Subset(val_ds, subset_val_indices)


train_loader = DataLoader(train_dataset_small, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset_small, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)

print(f"Train set size: {len(train_ds)}, Val set size: {len(val_ds)}, Test set size: {len(test_ds)}")

In [5]:
pose_ch = train_ds[0]['src_pose'].shape[0]
netG = DeformableGenerator(in_ch=3, pose_ch=pose_ch).to(device)
netD = PatchDiscriminator(in_ch_img=3, pose_ch=pose_ch).to(device)

print(netG)
print(netD)

DeformableGenerator(
  (enc1): DownsampleBlock(
    (conv): ConvBlock(
      (main): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): ReLU(inplace=True)
      )
    )
  )
  (enc2): DownsampleBlock(
    (conv): ConvBlock(
      (main): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
  )
  (enc3): DownsampleBlock(
    (conv): ConvBlock(
      (main): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
  )
  (enc4): DownsampleBlock(
    (conv): ConvBlock(
      (main): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), st

In [6]:
ckpt = torch.load("outputs/ckpts/checkpoint_latest.pth")
netG.load_state_dict(ckpt['netG'])
netD.load_state_dict(ckpt['netD'])

/tmp/ipykernel_1103730/334369987.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("outputs/ckpts/checkpoint_latest.pth")


<All keys matched successfully>

In [ ]:
train_model(
    netG, netD,
    train_loader, val_loader,
    device=device,
    num_epochs=10,
    lr=lr/3,
    betas=betas,
    lambda_nn=lambda_nn,
    lambda_p=lambda_perceptual,
    lambda_off=lambda_offset,
    use_perceptual=use_perceptual,
    checkpoint_dir=ckpt_dir,
    sample_dir=sample_dir,
    num_gen_updates=2
)

In [12]:
from utils.checkpoint import save_checkpoint 
checkpoint_dir="outputs/ckpts"

ckpt = {
            'epoch': 7,
            'netG': netG.state_dict(),
            'netD': netD.state_dict(),
            'optimG': "",
            'optimD': "",
        }

save_checkpoint(ckpt, checkpoint_dir, filename="checkpoint_latest.pth")

Saved checkpoint: outputs/ckpts/checkpoint_latest.pth


In [15]:
infer_model(netG, test_loader, device=device, output_dir=infer_out_dir)

Inference done; results saved to outputs/infer
